## BLEU: BiLingual Evaluation Understudy

*NLP evaluation metric used in Machine Translation tasks*

*Suitable for measuring corpus level similarity*

*$n$-gram comparison between words in candidate sentence and reference sentences*

*Range: 0 (no match) to 1 (exact match)*

### 1. Libraries
*Install and import necessary libraries*


In [1]:
import nltk
import nltk.translate.bleu_score as bleu

import math
import numpy
import os

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### 2. Dataset
*Array of words: candidate and reference sentences split into words*

In [2]:
hyp = "/content/real_data.txt"
ref_a = "/content/genspam_seq.txt"
ref_b = "/content/genspam_sto.txt"

### 3. *Sentence* score calculation
*Compares 1 hypothesis (candidate or source sentence) with 1+ reference sentences, returning the highest score when compared to multiple reference sentences.*

In [8]:
score_ref_a = bleu.sentence_bleu([ref_a], hyp)
print("Hyp and ref_a are the same: {}".format(score_ref_a))
score_ref_b = bleu.sentence_bleu([ref_b], hyp)
print("Hyp and ref_b are different: {}".format(score_ref_b))
score_ref_ab = bleu.sentence_bleu([ref_a, ref_b], hyp)
print("Hyp vs multiple refs: {}".format(score_ref_ab))

Hyp and ref_a are the same: 0.7400828044922853
Hyp and ref_b are different: 8.190757052088229e-155
Hyp vs multiple refs: 0.7400828044922853


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


### 4. *Corpus* score calculation
*Compares 1 candidate document with multiple sentence and 1+ reference documents also with multiple sentences.*

* Different than averaging BLEU scores of each sentence, it calculates the score by *"summing the numerators and denominators for each hypothesis-reference(s) pairs before the division"*

In [4]:
score_ref_a = bleu.corpus_bleu([[ref_a]], [hyp])
print("1 document with 1 reference sentence: {}".format(score_ref_a))
score_ref_a = bleu.corpus_bleu([[ref_a, ref_b]], [hyp])
print("1 document with 2 reference sentences: {}".format(score_ref_a))
score_ref_a = bleu.corpus_bleu([[ref_a], [ref_b]], [hyp, hyp])
print("2 documents with 1 reference sentence each: {}".format(score_ref_a))

1 document with 1 reference sentence: 1.0
1 document with 2 reference sentences: 1.0
2 documents with 1 reference sentence each: 0.8778107713916036


### 5. BLEU-$n$
*In BLEU-$n$, $n$-gram scores can be obtained in both **sentence** and **corpus** calculations and they're indicated by the **weights** parameter.*

* *weights*: length 4, where each index contains a weight corresponding to its respective $n$-gram.
* $n$-gram with $n \in \{1, 2, 3, 4\}$
* $\textit{weights}=(W_{N=1}, W_{N=2}, W_{N=3}, W_{N=4})$



In [ ]:
score_1gram = bleu.sentence_bleu([ref_b], hyp, weights=(1,0,0,0))
score_2gram = bleu.sentence_bleu([ref_b], hyp, weights=(0,1,0,0))
score_3gram = bleu.sentence_bleu([ref_b], hyp, weights=(0,0,1,0))
score_4gram = bleu.sentence_bleu([ref_b], hyp, weights=(0,0,0,1))
print("N-grams: 1-{}, 2-{}, 3-{}, 4-{}".format(score_1gram, score_2gram, score_3gram, score_4gram))

* Cumulative N-grams: *by default, the score is calculatedby considering all $N$-grams equally in a geometric mean*

In [6]:
score_ngram1 = bleu.sentence_bleu([ref_b], hyp)
score_ngram = bleu.sentence_bleu([ref_b], hyp, weights=(0.25,0.25,0.25,0.25))
score_ngram_geo = (11/11*9/10*6/9*4/8)**0.25
print("N-grams: {} = {} = ".format(score_ngram1, score_ngram, score_ngram_geo))

N-grams: 0.7400828044922853 = 0.7400828044922853 = 


### Further testing

In [7]:
hyp = str('she read the book because she was interested in world history').split()
ref_a = str('she was interested in world history because she read the book').split()
hyp_b = str('the book she read was about modern civilizations.').split()
ref_b = str('the book she read was about modern civilizations.').split()

score_a = bleu.sentence_bleu([ref_a], hyp)
score_b = bleu.sentence_bleu([ref_b], hyp_b)
score_ab = bleu.sentence_bleu([ref_a], hyp_b)
score_ba = bleu.sentence_bleu([ref_b], hyp)
score_ref_a = bleu.corpus_bleu([[ref_a], [ref_b]], [hyp, hyp_b])
average = (score_a+score_b)/2
corpus = math.pow((11+8)/19 * (9+7)/(17) * (6+6)/(9+6) * (4+5)/(8+5), 1/4)
print("Sent: {}, {}, {}, {} - Corpus {}, {}, {}".format(score_a, score_b, score_ab, score_ba, score_ref_a, average, corpus))

Sent: 0.7400828044922853, 1.0, 6.664457123729399e-155, 8.190757052088229e-155 - Corpus 0.8496988908521796, 0.8700414022461427, 0.8496988908521795


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


**Jaccard Diversity Calculation**

In [ ]:
def Jaccard_Diversity(doc1, doc2): 
    
    # List the unique words in a document
    words_doc1 = set(doc1.lower().split()) 
    words_doc2 = set(doc2.lower().split())
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)

In [ ]:
print(1-Jaccard_Diversity(hyp,ref_a))